In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## SIRV Epidemic Model

#### S = susceptible
#### I = infected
#### R = recovered
#### V = vaccinated

* $\beta$ = number infections per infected

* $\gamma$ = recovery rate

* $\frac{1}{\mu}$ = average lifespan

* $v$ = vaccination rate

* $\frac{1}{\sigma}$ = average time before immunity loss from vaccine

* $\frac{1}{\omega}$ = average time before immunity loss from infection

* $\lambda$ = birth rate
 
----------------------
For simplicity we assume that $\lambda = \mu$

$
\Large\frac{dS}{dt} = (\sigma V + \omega R) - \beta \cdot I \cdot \frac{S}{N} - vS \\
\Large\frac{dV}{dt} = vS - \sigma V\\
\Large\frac{dI}{dt} = \beta \cdot I \cdot \frac{S}{N} - \gamma I \\
\Large\frac{dR}{dt} = \gamma I - \omega R
$

In [11]:
def sim_SIR(N, I, vday, maxdays, vrate=0.5, gamma=0.2, beta=1.2, sigma=1/100, omega=1/150):
    S = N
    V = 0
    I = I
    R = 0
    S_all = []
    V_all = []
    I_all = []
    R_all = []

    # the model
    for day in range(0, maxdays):
        if day < vday:
            dS = 0.001*N + (omega*R) - (beta * I * (S / N))
            dV = 0
            dI = (beta * I * (S/N) - (gamma * I))
            dR = (gamma*I - omega*R)
        elif day >= vday:
            dS = 0.001*N + (sigma*V + omega*R) - (beta * I * (S / N)) - vrate*S
            dV = (vrate*S - sigma*V)
            dI = (beta * I * (S/N) - (gamma * I))
            dR = (gamma*I - omega*R)

        if (S+dS) < 0: 
            S = 0      
        else: 
            S += dS

        I += dI
        R += dR
        V += dV
        
        S_all.append(S)
        I_all.append(I)
        R_all.append(R)
        V_all.append(V)
    
    return S_all, I_all, R_all, V_all

In [13]:
def plot_SIR(N, I, vday, maxdays, vrate, gamma, beta, sigma, omega):
    S_all, I_all, R_all, V_all = sim_SIR(N, I, vday, maxdays, vrate, gamma, beta, sigma, omega)
    x = range(0, maxdays)
    plt.figure(figsize=[14,4])
    plt.fill_between(x, S_all, color='tab:purple', alpha=0.25, label='Susceptible')
    plt.fill_between(x, V_all, color='tab:green', alpha=0.25, label='Vaccinated')
    plt.fill_between(x, I_all, color='tab:red', alpha=0.25, label='Infected')
    plt.fill_between(x, R_all, color='tab:blue', alpha=0.25, label='Recovered')
    plt.plot(S_all, color='tab:purple', lw=2)
    plt.plot(V_all, '--', color='tab:green', lw=2)
    plt.plot(I_all, color='tab:red', lw=2)
    plt.plot(R_all, color='tab:blue', lw=2)
    plt.xlabel('Days')
    plt.ylabel('Population')

interact(plot_SIR,
         N=widgets.IntSlider(min=100, max=10000000, step=100, value=10000000),
         I=widgets.IntSlider(min=1, max=100, step=1, value=1),
         vday=widgets.IntSlider(step=1, max=1000, value=365),
         maxdays=widgets.IntSlider(min=10, max=5000, step=1, value=500),
         vrate=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05),
         gamma=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.12),
         beta=widgets.FloatSlider(min=0.0, step=0.01, value=0.35),
         sigma=widgets.FloatSlider(min=0.0, max=1.0, step=0.005, value=1/100),
         omega=widgets.FloatSlider(min=0.0, max=1.0, step=0.005, value=1/150))

interactive(children=(IntSlider(value=10000000, description='N', max=10000000, min=100, step=100), IntSlider(v…

<function __main__.plot_SIR(N, I, vday, maxdays, vrate, gamma, beta, sigma, omega)>

## Stability analysis
